In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [6]:
pd.read_sql(
    """
    SELECT * FROM boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
pd.read_sql(
    """
    SELECT 
    count(NUllIF(null,CRIM)) as CRIM_N, 
    count(NUllIF(null,ZN)) as ZN_N, 
    count(NUllIF(null,INDUS)) as INDUS_N, 
    count(NUllIF(null,CHAS)) as CHAS_N, 
    count(NUllIF(null,NOX)) as NOX_N
    FROM boston
    """,
    con,
)

,CRIM_N,ZN_N,INDUS_N,CHAS_N,NOX_N
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
pd.read_sql(
    """
    SELECT 
    count(distinct CRIM) as CRIM_c, 
    count(distinct ZN) as ZN_c, 
    count(distinct INDUS) as INDUS_c, 
    count(distinct CHAS) as CHAS_c, 
    count(distinct NOX) as NOX_c
    FROM boston
    """,
    con,
)

,CRIM_c,ZN_c,INDUS_c,CHAS_c,NOX_c
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
pd.read_sql(
    """
    with tab as
    (Select
    distinct
    'CRIM' Pok,
    avg(crim) over() med,
    min_crim min
    from
    (SELECT 
    crim,
    min(crim) over() min_crim,
    row_number() over(order by crim asc) rnac,
    count() over() lenn
    FROM boston)
    where rnac in (lenn/2, lenn/2+1)
    union all
    Select
    distinct
    'NOX' Pok,
    avg(NOX) over() med,
    min_NOX min
    from
    (SELECT 
    NOX,
    min(NOX) over() min_NOX,
    row_number() over(order by NOX asc) rnan,
    count() over() lenn
    FROM boston)
    where rnan in (lenn/2, lenn/2+1)
    union all
    Select
    distinct
    'ZN' Pok,
    avg(ZN) over() med,
    min_ZN min
    from
    (SELECT 
    ZN,
    min(ZN) over() min_ZN,
    row_number() over(order by ZN asc) rnaz,
    count() over() lenn
    FROM boston)
    where rnaz in (lenn/2, lenn/2+1)
    union all
    Select
    distinct
    'INDUS' Pok,
    avg(INDUS) over() med,
    min_INDUS min
    from
    (SELECT 
    INDUS,
    min(INDUS) over() min_INDUS,
    row_number() over(order by INDUS asc) rnai,
    count() over() lenn
    FROM boston)
    where rnai in (lenn/2, lenn/2+1)
    union all
    Select
    distinct
    'CHAS' Pok,
    avg(CHAS) over() med,
    min_CHAS min
    from
    (SELECT 
    CHAS,
    min(CHAS) over() min_CHAS,
    row_number() over(order by CHAS asc) rnaca,
    count() over() lenn
    FROM boston)
    where rnaca in (lenn/2, lenn/2+1)
    union all
    Select
    distinct
    'NOX' Pok,
    avg(NOX) over() med,
    min_NOX min
    from
    (SELECT 
    NOX,
    min(NOX) over() min_NOX,
    row_number() over(order by NOX asc) rnacn,
    count() over() lenn
    FROM boston)
    where rnacn in (lenn/2, lenn/2+1))
    Select * 
    from tab 
    where med=min
    """,
    con,
)

,Pok,med,min
0,ZN,0.0,0.0
1,CHAS,0.0,0.0


Вывод: Медиана = минимальному значению в столбцах ZN и CHAS, которые заполнены одним значением -0.00

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
pd.read_sql(
    """
    Select
    avg_1-avg_25,
    avg_1-avg_100,
    avg_1-avg_200,
    avg_1-avg_300
    from
    (select  
    'diff' uslov,
    avg(rm) avg_1
    from 
   (SELECT 
    rm,
    medv,
    dense_rank() over(order by medv desc) rang,
    row_number() over(order by medv asc) num
    FROM boston) where rang<2) a
    
  left join
  (Select 
   'diff' uslov,
   avg(rm) avg_25 
   from
   (SELECT 
    rm,
    medv,
    dense_rank() over(order by medv desc) rang,
    row_number() over(order by medv asc) num
    FROM boston)
  where num<26) b
  on a.uslov=b.uslov
  left join
  (Select 
   'diff' uslov,
   avg(rm) avg_100 
   from
   (SELECT 
    rm,
    medv,
    dense_rank() over(order by medv desc) rang,
    row_number() over(order by medv asc) num
    FROM boston)
  where num<100) c
  on a.uslov=c.uslov
  left join
  (Select 
   'diff' uslov,
   avg(rm) avg_200 
   from
   (SELECT 
    rm,
    medv,
    dense_rank() over(order by medv desc) rang,
    row_number() over(order by medv asc) num
    FROM boston)
  where num<200) d
  on a.uslov=d.uslov
  left join
  (Select 
   'diff' uslov,
   avg(rm) avg_300 
   from
   (SELECT 
    rm,
    medv,
    dense_rank() over(order by medv desc) rang,
    row_number() over(order by medv asc) num
    FROM boston)
  where num<300) e
  on a.uslov=e.uslov

    """,
    con,
)

,avg_1-avg_25,avg_1-avg_100,avg_1-avg_200,avg_1-avg_300
0,1.73616,1.604636,1.569744,1.511502


Вывод: средняя стоимость квартир находится в прямой зависимости с количеством комнат, с увеличением стоимости увеличивается и количество комнат, но незначительно.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [16]:
pd.read_sql(
    """
    select 
    * 
    from 
    (SELECT 
    lstat,
    medv,
    dense_rank() over(order by lstat) rang_lstat,
    dense_rank() over(order by medv desc) rang_medv
    FROM boston)
    where rang_medv=1
    """,
    con,
)

,lstat,medv,rang_lstat,rang_medv
0,1.73,50.0,1,1
1,1.92,50.0,2,1
2,2.88,50.0,6,1
3,2.96,50.0,8,1
4,2.97,50.0,9,1
5,3.16,50.0,14,1
6,3.26,50.0,15,1
7,3.32,50.0,16,1
8,3.70,50.0,23,1
9,3.73,50.0,24,1


Вывод: Это свидетельствует, что в домах с высокой стоимостью более низкий % низкого статуса населения - в основном до 4%. Но есть три дома, где этот показатель около 8%.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
pd.read_sql(
    """
    SELECT 
    distinct
    CHAS,
    avg(medv) over(partition by chas)
    FROM boston
    """,
    con,
)

,CHAS,avg(medv) over(partition by chas)
0,0.0,22.093843
1,1.0,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [53]:
pd.read_sql(
    """
    Select
    chas,
    case when krim=m_k then krim else null end crim,
    case when zhi=m_z then zhi else null end zn,
    case when busi=m_b then busi else null end indus
    from
    (With tab as
    (SelECT
    distinct
    chas,
    avg(crim) over(partition by chas) krim,
    avg(zn) over(partition by chas) zhi,
    avg(indus) over(partition by chas) busi
    FROM boston)
    
    Select 
    *,
    max(krim) over() m_k,
    max(zhi) over() m_z,
    max(busi) over() m_b
    from
    tab)
 where chas=1
    """,
    con,
)

,chas,crim,zn,indus
0,1.0,None,None,12.719143


Вывод: у реки больше коэффициент промышленной застройки, но ниже коэффициент криминала и жилой застройки, чем не у реки

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [91]:
pd.read_sql(
    """
    SELECT 
    distinct
    avg(nox) OVER() ag_nox,
    avg(indus) OVER() ag_indus
    FROM boston
    """,
    con,
)

,ag_nox,ag_indus
0,0.554695,11.136779


In [95]:
pd.read_sql(
    """
    Select 
    * ,
    narast_i/narast_n
    from
    (Select
    distinct
   a_i,
   ntile_indus,
   avg(a_i) over(order by ntile_indus RANGE UNBOUNDED PRECEDING) narast_i
    from
    (Select
    distinct
    avg(nox) over(partition by ntile_nox) a_n,
    avg(indus) over(partition  by ntile_indus) a_i,
    ntile_nox,
    ntile_indus
    from
    (SELECT 
    nox,
    INDUS,
    NTILE(10) OVER(ORDER BY nox asc) ntile_nox,
    NTILE(10) OVER(ORDER BY indus asc) ntile_indus
    FROM boston))) a,
    (Select
    distinct
   a_n,
   ntile_nox,
   avg(a_n) over(order by ntile_nox RANGE UNBOUNDED PRECEDING) narast_n
    from
    (Select
    distinct
    avg(nox) over(partition by ntile_nox) a_n,
    avg(indus) over(partition  by ntile_indus) a_i,
    ntile_nox,
    ntile_indus
    from
    (SELECT 
    nox,
    INDUS,
    NTILE(10) OVER(ORDER BY nox asc) ntile_nox,
    NTILE(10) OVER(ORDER BY indus asc) ntile_indus
    FROM boston))) b
    where a.ntile_indus=b.ntile_nox
    
    """,
    con,
)

,a_i,ntile_indus,narast_i,a_n,ntile_nox,narast_n,narast_i/narast_n
0,2.016078,1,2.016078,0.407692,1,0.407692,4.945100
1,3.645294,2,2.966454,0.434153,2,0.422125,7.027426
2,5.246275,3,3.636990,0.453443,3,0.431912,8.420670
3,6.471765,4,4.281257,0.493157,4,0.448615,9.543271
4,8.256078,5,4.892768,0.520118,5,0.463937,10.546186
5,10.749216,6,5.837356,0.548471,6,0.476745,12.244182
6,17.245800,7,7.687374,0.591440,7,0.491837,15.629932
7,18.100000,8,8.703240,0.633540,8,0.508314,17.121785
8,18.307200,9,9.929277,0.694060,9,0.520428,19.079072
9,21.938800,10,10.649849,0.780300,10,0.541217,19.677577


Вывод: Зависимость между загрязнением атмосферы и долей промышленной застройки прямая. При увеличении плотности застройки, концентрация увеличивается, но незначительно